<a href="https://colab.research.google.com/github/humairahs/geospatial-analysis-overpass-geopandas/blob/main/Overpass_turbo_%26_GeoPandas_for_geospatial_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overpass Turbo & GeoPandas
Integrating these 2 tools can benefit us to do geospatial analysis. It can also be extended for feature engineering task & implemented in many domains.
This example picks Surabaya, Indonesia (lat, lon:  -7.25, 112.768) & with EPSG:4326 coordinate system.

In [ ]:
!pip install folium
!pip install mapclassify

import requests
import geopandas as gpd
from shapely.geometry import Point, shape
import pandas as pd
import folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 882.2/882.2 kB 12.8 MB/s eta 0:00:00


In [ ]:
def get_overpass_data(latitude, longitude, tags, radius=500):
    overpass_url = "http://overpass-api.de/api/interpreter"
    tag_filter = ''.join([f'["{k}"="{v}"]' for k, v in tags.items()])

    query = f"""
    [out:json];
    (
      node{tag_filter}(around:{radius},{latitude},{longitude});
      way{tag_filter}(around:{radius},{latitude},{longitude});
    );
    out center;
    """

    response = requests.get(overpass_url, params={'data': query})
    # response.raise_for_status()
    data = response.json()

    elements = data.get('elements', [])
    if not elements:
        return gpd.GeoDataFrame(columns=['name', 'geometry', 'type'], geometry='geometry', crs="EPSG:4326")

    names = []
    geometries = []
    types = []

    for elem in elements:
        tags_elem = elem.get('tags', {})
        names.append(tags_elem.get('name', 'Unnamed'))
        types.append(elem.get('type', 'unknown'))

        if elem['type'] == 'node':
            lon_e = elem['lon']
            lat_e = elem['lat']
        else:
            center = elem.get('center')
            if center is None:
                continue
            lon_e = center['lon']
            lat_e = center['lat']
        geometries.append(Point(lon_e, lat_e))

    gdf = gpd.GeoDataFrame({'name': names, 'type': types}, geometry=geometries, crs="EPSG:4326")
    return gdf

In [ ]:
# Coordinates (latitude & longitude) of area we want to inspect
lat = -7.25
lon = 112.76
radius = 2000

# Various points of interest (POI) to be extracted from Overpass Turbo. It is returned as geodataframe type. The full available keys can be found in https://wiki.openstreetmap.org/wiki/Key:amenity
college = get_overpass_data(lat, lon, {"amenity": "college"}, radius=radius)
college['poi_type'] = 'college'
school = get_overpass_data(lat, lon, {"amenity": "school"}, radius=radius)
school['poi_type'] = 'school'
university = get_overpass_data(lat, lon, {"amenity": "university"}, radius=radius)
university['poi_type'] = 'university'

In [ ]:
# Compiled geodataframe of all POIs

poi_gdf = pd.concat([college, school, university])
poi_gdf = poi_gdf[poi_gdf['name'] != 'Unnamed']
poi_gdf.reset_index(drop=True, inplace=True)

poi_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   name      112 non-null    object  
 1   type      112 non-null    object  
 2   geometry  112 non-null    geometry
 3   poi_type  112 non-null    object  
dtypes: geometry(1), object(3)
memory usage: 3.6+ KB


# Example of use with dummy data
The dummy data 'dummy_num_students_data.csv' contains the number of students of all schools/colleges/universities. In real life, it can vary much with richer information.

In dummy data df assumed that the name of each object/POI is unique (in more standard way, we must ensure by typically using certain unique ID). This will be used as a key to merge with the POI gdf. It will allow us to spatially visualize the data points and derive relevant analysis from it.


In [ ]:
df = pd.read_csv('dummy_num_students_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112 entries, 0 to 111
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   num_of_students  112 non-null    int64 
 1   name             112 non-null    object
dtypes: int64(1), object(1)
memory usage: 1.9+ KB


In [ ]:
poi_gdf = poi_gdf.merge(df, on='name', how='left')

In [ ]:
poi_gdf.head()

,name,type,geometry,poi_type,num_of_students
0,Akademi Keperawatan Adi Husada,way,POINT (112.75124 -7.24284),college,660
1,Politeknik Kesehatan Kementerian Kesehatan Sur...,way,POINT (112.76024 -7.26635),college,583
2,Sekolah Tinggi Kesehatan Yayasan RSUD Dr Soetomo,way,POINT (112.76021 -7.26728),college,678
3,SD Kristen Buah Sulung,node,POINT (112.75678 -7.25773),school,783
4,SD/MI Hidayatul Wathon,node,POINT (112.74539 -7.24956),school,572


In [ ]:
def spatial_visualization(data, column1, tool_tip=None, save_result=False, categorical=False):
    m = folium.Map(location=[lat, lon], zoom_start=13)

    data.explore(
        column=column1,
        cmap='coolwarm',
        scheme='Quantiles',
        marker_kwds = {'radius':10},
        name=f'{column1} Layer',
        legend=True,
        m=m,
        categorical=categorical,
    )

    folium.TileLayer('https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}', control=True, attr='Google').add_to(m)
    folium.LayerControl().add_to(m)
    if save_result:
        m.save(outfile='prediction_and_real_values.html')

    return m

In [ ]:
# Test to visualize only the school POI type and the pattent of number of students distribution in the area
a = poi_gdf[poi_gdf['poi_type']=='school']

spatial_visualization(a, 'num_of_students')